In [1]:
from res.workflow import WindOnshoreWorkflow
from res import windpower

import geokit as gk
import pandas as pd
import numpy as np
from os.path import join

import matplotlib.pyplot as plt
%matplotlib inline

---
# Set data paths & constants

In [2]:
TESTDATA = join("..","testing","data")
MERRA_SOURCE = join(TESTDATA,"merra-like.nc4")
LAND_COVER_SOURCE = join(TESTDATA,"clc-aachen_clipped.tif")
GWA_SOURCE = join(TESTDATA,"gwa50-like.tif")
PLACEMENT_SOURCE = join(TESTDATA,"turbinePlacements.shp")

TURBINE = "V136-3450"
CAPACITY = 3000
ROTORDIAM = 120
HUB_HEIGHT = 82

BATCH_SIZE = 20000 # Total number of turbines to simulate at once (best value depends on your pc)
JOBS = 4 # The number of concurrent jobs
VERBOSE = True # When True, progress will be printed as the function executes

---
# Simulate explicit locatons with a constant turbine and hub height

In [14]:
placements = [(5.98520,50.79725),
              (5.99469,50.79421),
              (5.99411,50.80094),
              (6.00475,50.78443),
              (6.00417,50.79116),
              (6.00360,50.79789),
              (6.01309,50.79485),
              (6.01251,50.80158),
              (6.02200,50.79853),
              (6.02142,50.80526),]

In [17]:
# Using a turine name from the TurbineLibrary
result = WindOnshoreWorkflow(placements, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, verbose=VERBOSE)

Starting at: 2017-12-06 12:18:35.983274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.00s
Convolving power curves at +0.06s
Initializing simulations at +0.94s
Simulating 1 groups at +0.94s
 0: Starting at +0.94s
 0: Finished 10 turbines +0.95s (1250.00 turbines/sec)
Finished simulating 10 turbines (10 surviving) at +0.95s (10.54 turbines/sec)
Done at +0.95s!


In [18]:
# Using a synthetic power curve
result = WindOnshoreWorkflow(placements, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             capacity=CAPACITY, rotordiam=ROTORDIAM, hubHeight=HUB_HEIGHT, verbose=VERBOSE)

Starting at: 2017-12-06 12:18:37.359274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.00s
Convolving power curves at +0.07s
Initializing simulations at +0.95s
Simulating 1 groups at +0.95s
 0: Starting at +0.95s
 0: Finished 10 turbines +0.95s (1111.11 turbines/sec)
Finished simulating 10 turbines (10 surviving) at +0.95s (10.47 turbines/sec)
Done at +0.95s!


---
# Simulate many locatons using multiple CPUs 

In [9]:
placements = np.column_stack([
    np.linspace(5.7,6.7,10000),  # 10,000 longitudes points
    np.linspace(50.4,51.0,10000) # 10,000 latitudes points
])


result = WindOnshoreWorkflow(placements, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, verbose=VERBOSE, 
                             batchSize=BATCH_SIZE, jobs=JOBS)

Starting at: 2017-12-06 11:19:03.653274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.08s
Convolving power curves at +1.19s
Initializing simulations at +2.10s
Simulating 4 groups at +2.17s
Finished simulating 10000 turbines (10000 surviving) at +4.74s (2111.93 turbines/sec)
Done at +4.74s!


---
# Simulate all locations in a placement file

In [3]:
result = WindOnshoreWorkflow(PLACEMENT_SOURCE, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, batchSize=BATCH_SIZE, jobs=JOBS, 
                             verbose=VERBOSE)

Starting at: 2017-12-06 11:13:59.643274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.05s
Convolving power curves at +1.14s
Initializing simulations at +2.05s
Simulating 4 groups at +2.13s
Finished simulating 560 turbines (560 surviving) at +3.65s (153.26 turbines/sec)
Done at +3.65s!


---
# Filter locations before simulating

In [8]:
# Load placements into a dataframe
placements = gk.vector.extractAsDataFrame(PLACEMENT_SOURCE, outputSRS="latlon")

# Do whatever filtering you want...
placements = placements[::2]

# Simulate...
result = WindOnshoreWorkflow(placements, MERRA_SOURCE, powerCurve=TURBINE, hubHeight=HUB_HEIGHT, 
                             minCF=CF_MIN, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE, 
                             batchSize=BATCH_SIZE, jobs=JOBS)

Starting at: 2017-12-06 12:12:28.555274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.01s
Convolving power curves at +1.10s
Initializing simulations at +2.02s
Simulating 4 groups at +2.08s
Finished simulating 280 turbines (280 surviving) at +3.53s (79.28 turbines/sec)
Done at +3.53s!


---
# Unique definitions for each turbine

In [5]:
# Give a turbine name to each placement
placements = pd.DataFrame()
placements["lon"] = np.linspace(5.7,6.7,10000)             # latitude column MUST be given as 'lat'
placements["lat"] = np.linspace(50.4,51.0,10000)           # longitude column MUST be given as 'lon'
placements["hubHeight"] = np.linspace(49,52,10000)         # hub height column can be given, but must be named as 'hubHeight'
placements["powerCurve"] = ["E-126_EP4", "V136-3450"]*5000 # turbine name column can be given, but must be named as either
                                                           #  'turbine' or 'powerCurve'
print("Placements:")
print(placements.head(10))

print("\nSimulation:")
result = WindOnshoreWorkflow(placements, MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE, 
                             batchSize=BATCH_SIZE, jobs=1, extract="production")


Placements:
      lon       lat  hubHeight powerCurve
0  5.7000  50.40000    49.0000  E-126_EP4
1  5.7001  50.40006    49.0003  V136-3450
2  5.7002  50.40012    49.0006  E-126_EP4
3  5.7003  50.40018    49.0009  V136-3450
4  5.7004  50.40024    49.0012  E-126_EP4
5  5.7005  50.40030    49.0015  V136-3450
6  5.7006  50.40036    49.0018  E-126_EP4
7  5.7007  50.40042    49.0021  V136-3450
8  5.7008  50.40048    49.0024  E-126_EP4
9  5.7009  50.40054    49.0027  V136-3450

Simulation:
Starting at: 2017-12-06 12:58:07.245274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.04s
Convolving power curves at +0.12s
   Convolving 2 power curves...
Initializing simulations at +3.01s
Simulating 1 groups at +3.01s
 0: Starting at +3.01s
 0: Finished 10000 turbines +8.90s (1698.66 turbines/sec)
Finished simulating 10000 turbines (10000 surviving) at +8.91s (1121.70 turbines/sec)
Done at +8.91s!


In [7]:
# Generate a synthetic pwoer curve for each placement
placements = pd.DataFrame()
placements["lon"] = np.linspace(5.7,6.7,10000)        # latitude column MUST be given as 'lat'
placements["lat"] = np.linspace(50.4,51.0,10000)      # longitude column MUST be given as 'lon'
placements["hubHeight"] = np.linspace(49,52,10000)    # hub height column must be named as 'hubHeight'
placements["capacity"] = np.linspace(3000,4000,10000) # capacity column must be named as 'capacity'
placements["rotordiam"] = np.ones(10000)*120          # rotor diameter column must be named as 'rotordiam'

print("Placements:")
print(placements.head(10))

print("\nSimulation:")
result = WindOnshoreWorkflow(placements, MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE, 
                             batchSize=BATCH_SIZE, jobs=JOBS, extract="production")

Placements:
      lon       lat  hubHeight    capacity  rotordiam
0  5.7000  50.40000    49.0000  3000.00000      120.0
1  5.7001  50.40006    49.0003  3000.10001      120.0
2  5.7002  50.40012    49.0006  3000.20002      120.0
3  5.7003  50.40018    49.0009  3000.30003      120.0
4  5.7004  50.40024    49.0012  3000.40004      120.0
5  5.7005  50.40030    49.0015  3000.50005      120.0
6  5.7006  50.40036    49.0018  3000.60006      120.0
7  5.7007  50.40042    49.0021  3000.70007      120.0
8  5.7008  50.40048    49.0024  3000.80008      120.0
9  5.7009  50.40054    49.0027  3000.90009      120.0

Simulation:
Starting at: 2017-12-06 13:02:06.569274
Arranging placements at +0.00s
Pre-loading windspeeds at +0.06s
Convolving power curves at +1.18s
   Convolving 9 power curves...
Initializing simulations at +5.58s
Simulating 4 groups at +5.64s
Finished simulating 10000 turbines (10000 surviving) at +9.48s (1054.85 turbines/sec)
Done at +9.48s!


---
# Various extraction methods
Options are:
* "raw"  -> Returns the complete time-series production for each simulated turbine
* "capacityFactor" -> Returns the average capacity factor for each simulated turbine
* "averageProduction" -> Returns the average production at each timestep
* "batch" -> Returns nothing, but if an output file is specified a "raw" output will be created for each batch

In [11]:
result = WindOnshoreWorkflow(PLACEMENT_SOURCE, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, batchSize=BATCH_SIZE, jobs=JOBS, 
                             verbose=False, extract="raw")
result.head()

,"(5.98520,50.79725)","(5.99469,50.79421)","(5.99411,50.80094)","(6.00475,50.78443)","(6.00417,50.79116)","(6.00360,50.79789)","(6.01309,50.79485)","(6.01251,50.80158)","(6.02200,50.79853)","(6.02142,50.80526)",...,"(6.31490,50.82850)","(6.31437,50.83523)","(6.31383,50.84197)","(6.31330,50.84870)","(6.32384,50.83216)","(6.32331,50.83889)","(6.32278,50.84563)","(6.33332,50.82909)","(6.33279,50.83582)","(6.33226,50.84255)"
2005-01-01 00:30:00+00:00,333.143636,456.659415,449.300331,268.295160,449.951125,449.951125,519.447598,441.608939,486.066782,495.696282,...,372.642277,382.672840,451.153515,451.153515,362.440942,403.687954,474.933402,275.720410,360.499203,398.658392
2005-01-01 01:30:00+00:00,255.830664,355.643447,349.683410,203.734104,350.245002,350.245002,406.541594,343.455542,379.510124,387.317733,...,287.500371,295.787849,351.184168,351.184168,279.452225,312.564942,370.448454,209.681996,277.885357,308.706178
2005-01-01 02:30:00+00:00,213.638552,300.207095,295.026439,168.719278,295.532409,295.532409,344.489530,289.614280,320.983460,327.777445,...,240.959574,248.238454,296.330834,296.330834,234.081508,262.684503,313.080905,173.833987,232.724366,259.442674
2005-01-01 03:30:00+00:00,150.818671,216.857026,212.879344,117.078475,213.290072,213.290072,250.945485,208.726644,232.857394,238.089481,...,171.429562,177.088051,213.880616,213.880616,166.315762,187.979322,226.751781,120.894019,165.284728,185.634343
2005-01-01 04:30:00+00:00,117.484271,171.934162,168.634013,90.060393,168.985547,168.985547,200.293097,165.190885,185.241061,189.595228,...,134.333233,139.045148,169.464542,169.464542,130.184900,147.964113,180.152073,93.143775,129.338177,146.090968


In [13]:
result = WindOnshoreWorkflow(PLACEMENT_SOURCE, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, batchSize=BATCH_SIZE, jobs=JOBS, 
                             verbose=False, extract="capacityFactor")
result.head()

(5.98520,50.79725)    0.335618
(5.99469,50.79421)    0.394444
(5.99411,50.80094)    0.391305
(6.00475,50.78443)    0.298416
(6.00417,50.79116)    0.391432
dtype: float64

In [12]:
result = WindOnshoreWorkflow(PLACEMENT_SOURCE, merra=MERRA_SOURCE, gwa=GWA_SOURCE, landcover=LAND_COVER_SOURCE,
                             powerCurve=TURBINE, hubHeight=HUB_HEIGHT, cfMin=CF_MIN, batchSize=BATCH_SIZE, jobs=JOBS, 
                             verbose=False, extract="averageProduction")
result.head()

2005-01-01 00:30:00+00:00    365.754992
2005-01-01 01:30:00+00:00    287.918736
2005-01-01 02:30:00+00:00    244.041949
2005-01-01 03:30:00+00:00    173.182568
2005-01-01 04:30:00+00:00    125.666624
dtype: float64

---
# Saving result as an output file
Options are:
* ".shp" -> As a shape file (only available when extracting the capacity factors)
* ".csv" -> As a csv file
* ".nc"  -> As a netCDF4 file

In [ ]:
result = windProductionFromMerraSource(PLACEMENT_SOURCE, MERRA_SOURCE, turbine=TURBINE, hubHeight=HUB_HEIGHT, 
                                       cfMin=CF_MIN, gwaSource=GWA_SOURCE, lcSource=LAND_COVER_SOURCE, 
                                       batchSize=BATCH_SIZE, jobs=JOBS, output="test.csv")